In [0]:

import delta


def table_exists(catalog, database, table):
  count = (spark.sql(f"SHOW TABLES FROM {catalog}.{database}")
           .filter(f"database = '{database}' AND tableName = '{table}' ")
           .count())
  
  return count == 1

In [0]:
catalog = "bronze"
schema = "f1_world"
tableName = dbutils.widgets.get("tableName")
output_streaming = "/Volumes/raw/f1_world/streaming_source/"
source_path = f"/Volumes/raw/{schema}/"
TARGET_RACEID = 1141
files_split = ["constructor_results" ,"constructor_standings", "driver standings", "lap_times", "pit_stops", "qualifying", "results", "sprint_results"]

In [0]:
if not table_exists(catalog, schema, tableName):
  print(f"Criando tabela {tableName}")

  if tableName in files_split:

    df_split = (spark.read
          .format("csv")
          .options(header="true")
          .load(f"{source_path}{tableName}/"))
    
    # Consolidando full load sem a corrida alvo
    df_batch = df_split.filter(df_split.raceId != TARGET_RACEID)
    (df_batch.coalesce(1)
            .write
            .format("delta")
            .mode("overwrite")
            .saveAsTable(f"{catalog}.{schema}.{tableName}"))

    print("Tabela delta criado com sucesso")
    
    # Separando streaming
    df_streaming = df_split.filter(df_split.raceId == TARGET_RACEID)
    (df_streaming.coalesce(1)
            .write
            .format("parquet")
            .mode("overwrite")
            .save(f"{output_streaming}{tableName}"))

  else:
    df_full = (spark.read
          .format("csv")
          .options(header="true")
          .load(f"{source_path}{tableName}/"))

    (df_full.coalesce(1)
            .write
            .format("delta")
            .mode("overwrite")
            .saveAsTable(f"{catalog}.{schema}.{tableName}"))
else:
  print(f"Tabela {tableName} ja existe")
                                        
    

In [0]:
%sql
drop table bronze.f1_world.lap_times